In [2]:
import json 
import numpy as np 
import os,sys 
import re 

In [ ]:
root_path="/home/user/xsj/LLMTaskPlanning/alfred_multiagent/data/json_2.1.0/"

root_path_len=len(root_path)
# with open(os.path.join(root_path,'../../scene_dict.json'),'w') as f :
#     f.write(json.dumps(scene_dict,sort_keys=False,indent=4,separators=(',',':')))
with open(os.path.join(root_path,'../../scene_dict.json'),'r') as f :
    scene_dict=json.load(f)

In [ ]:
# filepaths=[]
# for root, dirs,files in os.walk(root_path):
#     for filename in files:
#         if "json" in filename:
#             filepaths.append(os.path.join(root,filename)[root_path_len:])

In [3]:
from NL2HLTLtaskPlanner import utils as util

In [4]:
GPTinterface=util.GPTinterface(
            useFakeGPTAPI=(not True),
            readFromFormerLog=False,
            logger=None,
            TranslateNL2TLInterface=util.NL2TLTranslater(
                    machine_type='openaiAPI',
                    base_url = "127.0.0.1:8000"
                # communicatorWithServer=util.communicator.communicationViaSSH(
                #     connectCommand="mill19",
                #     serverFolderPath='/home/icl-mill19/xsj/model_weight/scp-folder',
                #     clientFolderPath='/home/icl-mill19/xsj/NL2HLTL/exp',
                #     fileName='prompt.txt',)
                    ),
            )

In [5]:
check_flag=True
trial_count=0
for scene_name in scene_dict.keys():
    for trial_path in np.sort(scene_dict[scene_name]):
        try:
            new_path=os.path.join(root_path,'../../task_wo_position/{}.json'.format(trial_path))
            if not os.path.exists(new_path):
                os.makedirs(os.path.dirname(new_path))
            else:
                # print('{} exists'.format(trial_path))
                continue
            with open(os.path.join(root_path,trial_path),'r') as f:
                trial=json.load(f)
            trial_names= [re.match("([a-zA-Z]+)",x["objectName"]).group(0) for x in trial["scene"]["object_poses"]]
            trial_tasks= [x["task_desc"] for x in trial["turk_annotations"]["anns"]]
            prompt=[{
                "role": "user",
                "content":"""according to the task {} and the breakdown of execution steps {}\n currently in the environment we have the following objects, please extract all objects that may be involved in completing this task {}. Please return in JSON
    ```json
    {{
        "relative_objects": [object 1, object 2, ...],
        "explanation":
    }},
    ```
    Please consider carefully and provide the most reasonable answer.
    """.format("; ".join(trial_tasks),"",", ".join(trial_names))
    }
                ]
            # print(prompt)

            ret=GPTinterface.communicate(prompt,task_stage='Q_envInfoExtraction',modelVersion='3',print_screen=False)
            jsonstr=util.splitJSONfromTXT(ret)[-1]
            # print(jsonstr)
            


            with open(new_path,'w') as f :
                f.write(json.dumps(json.loads(jsonstr),sort_keys=False,indent=4,separators=(',',':')))
            trial_count+=1
            print(trial_count,trial_path)
        except:
            pass
        # check_flag=False
        if check_flag==False:
            break
    if check_flag==False:
        break
    